In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import random
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
#ptimizer=tfa.optimizers.AdamW(lr = 2e-3, weight_decay = 1e-5, clipvalue = 700)
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.optimizers import SGD, Adam, RMSprop

import tensorflow_addons as tfa

from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.merge import concatenate


from sklearn.metrics import log_loss
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('../input/iterativestratification/')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
def seed_everything(seed=999):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(49)

In [ ]:
PATH = '../input/lish-moa/'

In [ ]:
train_features = pd.read_csv(PATH+'train_features.csv')
test_features =pd.read_csv(PATH+'test_features.csv')
train_targets_nonscored =pd.read_csv(PATH+'train_targets_nonscored.csv')
train_targets_scored =pd.read_csv(PATH+'train_targets_scored.csv')
sample_submission =pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
###################Select Feature#######################
from sklearn.ensemble import ExtraTreesClassifier

ID = ['sig_id']
CP = ['cp_type','cp_time','cp_dose']
G = [x for x in train_features.columns if 'g-' in x]
C = [x for x in train_features.columns if 'c-' in x]
X = train_features[C+G].values

Targets = train_targets_scored.columns[1:].tolist()
Y = train_targets_scored[Targets].values[:,1:]
model = ExtraTreesClassifier(n_estimators=100)
model.fit(X, Y)
print(model.feature_importances_)

In [ ]:
sel_feat= model.feature_importances_.argsort()[-200:][::-1]
Z=train_features[C+G]
name_sel_feat= Z.iloc[:,sel_feat].columns
###################Select Feature#######################

In [ ]:
#Let us check data types of train_features
train_features.select_dtypes(include=['object','category','int','float']).dtypes

In [ ]:
#Which are the object datatypes
train_features.select_dtypes(include=['object']).dtypes

In [ ]:
#This means we have have cp_type and cp_dose as categorical variables, also cp_time which has only three values 24, 48 and 72
tr_features= [col for col in train_features.columns if col!='sig_id']
print("Length of train features list without 'sig_id' is:", len(tr_features))

In [ ]:
# Now use label encoder to convert train_features and test_features df together
for train_feat in ['cp_type', 'cp_dose','cp_time']:
    le = LabelEncoder()
    le.fit(list(train_features[train_feat].astype(str).values) + list(test_features[train_feat].astype(str).values))
    train_features[train_feat] = le.transform(list(train_features[train_feat].astype(str).values))
    test_features[train_feat] = le.transform(list(test_features[train_feat].astype(str).values))

In [ ]:
#Let us keep on train_feat columns for NN
#
tr_features = name_sel_feat
train = train_features[tr_features]
test= test_features[tr_features]
train_target = train_targets_scored.drop(['sig_id'], axis=1)
train_target_aux = train_targets_nonscored.drop(['sig_id'], axis=1)

In [ ]:
#So label encoding is done, let us move towards building NN
#We are using StandardScalar to transform all features and feed a balanced input to the neural net

sc = StandardScaler()
#train = sc.fit_transform(train)
#test = sc.transform(test)

In [ ]:
len_X =int(train.shape[1])
print(len_X)

In [ ]:
target_feat_len= train_target.shape[1]
target_aux_len= train_target_aux.shape[1]
#print(target_feat_len,train_target_aux)

In [ ]:
## You can play around with network architecture and check what works

#Using functional API, we will create a model with two outputs. First for scored columns and second for nonscored columns.
#Nonscored output is forcing model to consider this targets and I "hope" that it will produce more better accuracy
#Using batch normalization and high dropouts so that we prevent overfitting

#9/11 added tfa.layers.WeightNormalization

def getModel2(input_dim,target_feature_length,target_auxiliary_length):
    visible = Input(shape=(input_dim,))
    hidden1 = tfa.layers.WeightNormalization(Dense(875, activation='relu'))(visible)
    batchnorm1= BatchNormalization()(hidden1)
    dropout1= Dropout(0.5)(batchnorm1)
    hidden2 = tfa.layers.WeightNormalization(Dense(1750, activation='relu'))(dropout1)
    batchnorm2= BatchNormalization()(hidden2)
    dropout2= Dropout(0.5)(batchnorm2)
    hidden3 = tfa.layers.WeightNormalization(Dense(100, activation='relu'))(dropout2)
    batchnorm3= BatchNormalization(name="batchnorm3")(hidden3)
    dropout3= Dropout(0.5,name="dropout3")(batchnorm3)    
    output_2 = tfa.layers.WeightNormalization(Dense(target_auxiliary_length, activation='sigmoid',name="ouput_2"))(dropout3)
    
    hidden4 = tfa.layers.WeightNormalization(Dense(206, activation='relu',name="hidden4"))(output_2)
    batchnorm4= BatchNormalization(name="batchnorm4")(hidden4)
    dropout4= Dropout(0.5,name="dropout4")(batchnorm4)
    
    concat1 = tf.keras.layers.Concatenate(name="concate_nonscored_feat")([output_2, dropout4])
    
    output1 = tfa.layers.WeightNormalization(Dense(target_feature_length, activation='sigmoid', name="outputscore"))(concat1)
    output2 = tfa.layers.WeightNormalization(Dense(target_auxiliary_length, activation='sigmoid',name="outputnonscore"))(concat1)
    model = Model(inputs=visible, outputs=[output1,output2])
    return model

In [ ]:
#Create a sample model and check summay
modelNN= getModel2(len_X,target_feat_len,target_aux_len)
modelNN.summary()

In [ ]:
def metric(y_true, y_pred):
    #print('y_true:', y_true.head(3))
    #print('y_pred:',y_pred.head(3))
    metrics = []
    for _target in train_target.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)


In [ ]:
#Plot model and delete from memory
plot_model(modelNN, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
del modelNN

In [ ]:
from sklearn.model_selection import KFold
#from sklearn.model_selection import MultilabelStratifiedKFold
NFOLD = 10

BATCH_SIZE=16
EPOCHS=100

pout = np.zeros((test_features.shape[0], target_feat_len))
paux = np.zeros((test_features.shape[0], target_aux_len))
foldmetric=0
#Train is already a numpy as it went through StandardScalar
train_features = train
#train_targets needs to be converted to numpy
train_targets = train_target
train_target_aux_np=train_target_aux

pred = np.zeros((train_features.shape[0], target_feat_len))

cnt=0
kf = KFold(n_splits=NFOLD)
#kf = MultilabelStratifiedKFold(n_splits=NFOLD,shuffle=True, random_state=49)
for tr_idx, val_idx in kf.split(train_features):
#for tr_idx, val_idx in enumerate(kf.split(train_targets,train_targets)):
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=0.0001, mode='auto')
    cnt += 1
    print(f"FOLD {cnt}")
    #net = getModel(len_X,target_feat_len)
    net= getModel2(len_X,target_feat_len,target_aux_len)
    #net.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    net.compile(optimizer = tfa.optimizers.AdamW(lr = 2e-3, weight_decay = 1e-5, clipvalue = 700), loss = 'binary_crossentropy', metrics = ['accuracy'])
    net.fit(train_features.iloc[tr_idx], [train_targets.iloc[tr_idx],train_target_aux_np.iloc[tr_idx]], batch_size=BATCH_SIZE, epochs=EPOCHS, 
            validation_data=(train_features.iloc[val_idx], [train_targets.iloc[val_idx],train_target_aux_np.iloc[val_idx]]), verbose=0, callbacks=[early_stopping,reduce_lr_loss])
    print("Training for this fold", net.evaluate(train_features.iloc[tr_idx], [train_targets.iloc[tr_idx],train_target_aux_np.iloc[tr_idx]], verbose=0, batch_size=BATCH_SIZE))
    print("Validation for this fold", net.evaluate(train_features.iloc[val_idx], [train_targets.iloc[val_idx],train_target_aux_np.iloc[val_idx]], verbose=0, batch_size=BATCH_SIZE))
    print("Predict on validation data for this fold")
    pred[val_idx] = net.predict(train_features.iloc[val_idx], batch_size=BATCH_SIZE, verbose=0)[0]
    fold_metric = metric(train_targets.iloc[val_idx], pd.DataFrame(pred[val_idx],columns=train_target.columns))
    print(f'OOF Metric log_loss for this FOLD {cnt} : {fold_metric}')
    foldmetric += fold_metric/ NFOLD
    print("Average metric is:",foldmetric)
    #print(f'OOF Metric log_loss for this FOLD {cnt} : {metric(train_targets[val_idx], pred[val_idx])}')
    print("Predict test for nonscored targets")
    paux += net.predict(test, batch_size=BATCH_SIZE, verbose=0)[1] / NFOLD
    print("Predict test with scored targets")
    pout += net.predict(test, batch_size=BATCH_SIZE, verbose=0)[0] / NFOLD

In [ ]:
submission = pd.DataFrame(data=pout, columns=train_target.columns)
subpaux = pd.DataFrame(data=paux, columns=train_target_aux.columns)

In [ ]:
submission.insert(0, column = 'sig_id', value=sample_submission['sig_id'])
subpaux.insert(0, column = 'sig_id', value=sample_submission['sig_id'])

In [ ]:
submission.loc[test_features['cp_type'] == 1, train_target.columns] = 0

In [ ]:
submission.to_csv('submission.csv', index=False)
subpaux.to_csv('subaux.csv', index=False)

In [ ]:
print(submission.shape)
print(subpaux.shape)